In [18]:
from bs4 import BeautifulSoup as bs
import selenium


In [30]:
driver = webdriver.Chrome('C:/webdriver/bin/chromedriver.exe')
driver.get('https://www.mohap.gov.ae/en/Pages/profdoctors.aspx')
elem = driver.find_element_by_name('ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$ddlMedicalDistrict')
elem.send_keys('Dubai')
elem = driver.find_element_by_name('ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$ddlSpeciality')
elem.send_keys('Dental General Practitioner - Dentist')
elem = driver.find_element_by_name('ctl00$ctl54$g_358d628f_a927_4218_9168_f21c190735a7$ctl00$btnSearch')
elem.click()
driver.implicitly_wait(20) # seconds


In [42]:
source = driver.page_source



AttributeError: 'str' object has no attribute 'to_csv'

In [56]:
soup = bs(source,'lxml')
tag =soup.find_all('div','Drrow-prof col-xs-12 col-sm-12 col-md-6 col-lg-6')
tag[0].text.strip()


'Afra Abdulaziz Bin Alshikh M Alkhazraji\n\n\nFacility Name :\nmuhaisnah health center\n\n\n\nMedical Title :\nDentist-Dental General Practitioner\n\n\n\nNationality :\nUnited Arab Emirates'